## Processing

In [1]:
import glob
import random
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

random.seed(2021)

In [2]:
img_height = 320
img_width = 320
batch_size = 8

train_data_dir = "../IMAGES/"

data = []
labels = []

metadata_df = pd.read_csv('../metadata.csv')
metadata_df['race'] = pd.Categorical(pd.factorize(metadata_df.race)[0])
metadata_df.head()

,filename,race,gender
0,5599.jpg,0,F
1,5004.jpg,0,F
2,5024.jpg,0,F
3,6114.jpg,0,F
4,6127.jpg,0,F


In [3]:
metadata_df['race'] = metadata_df['race'].replace(0, 'Asian')
metadata_df['race'] = metadata_df['race'].replace(1, 'Black')
metadata_df['race'] = metadata_df['race'].replace(2, 'Caucasian')
metadata_df['race'] = metadata_df['race'].replace(3, 'Hispanic')

In [4]:
metadata_df = metadata_df[(metadata_df['race'] == 'Black') |
                          ((metadata_df['race'] == 'Caucasian'))]

In [5]:
num_classes = len(metadata_df['race'].value_counts().index)
metadata_df['race'].value_counts()

Black        358
Caucasian    333
Name: race, dtype: int64

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
metadata_df['race'] = le.fit_transform(metadata_df['race'])
le.classes_

array(['Black', 'Caucasian'], dtype=object)

In [7]:
metadata_df['gender'].value_counts()

M    351
F    340
Name: gender, dtype: int64

In [8]:
for i in metadata_df['filename']:
    key  = i
    label = metadata_df.loc[metadata_df['filename'] == key].iloc[0]['race']
    labels.append(label)
    filepath = train_data_dir+i
    image=tf.keras.preprocessing.image.load_img(filepath, color_mode='rgb', target_size= (img_height,img_width))
    image=np.array(image)
    data.append(image)
    
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, random_state=42) 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42)

In [9]:
import pickle

test_data = (X_test, y_test)
pickle.dump(test_data, open('../test_data_dha_2_classes.pkl', 'wb'))

In [10]:
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
X_val = np.asarray(X_val)
y_val = np.asarray(y_val)

In [11]:
from matplotlib import pyplot as plt
plt.imshow(X_train[10], interpolation='nearest')
plt.show()
display(y_train[10])

<Figure size 640x480 with 1 Axes>

1

## Model creation

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, concatenate, add, GlobalAveragePooling2D, BatchNormalization, Input
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras.applications.densenet import DenseNet121
from classification_models.tfkeras import Classifiers
from tensorflow.keras.models import load_model

resnet34, preprocess_input = Classifiers.get('resnet50')

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
input_a = Input(shape=(img_height, img_width, 3))
base_model = resnet34(input_tensor=input_a, include_top=False, input_shape=(img_height, img_width,3), weights='imagenet')

x = GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(num_classes, name='dense_logits')(x)
output = layers.Activation('softmax', dtype='float32', name='predictions')(x)
model = Model(inputs=[input_a], outputs=[output])

adam_opt = Adam(lr=0.000001)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.1, patience=2, min_lr=1e-6, verbose=1)
model.compile(optimizer=adam_opt, loss=tf.losses.CategoricalCrossentropy(),
                    metrics=[
                        tf.keras.metrics.AUC(curve='ROC', name='ROC-AUC'),
                        tf.keras.metrics.AUC(curve='PR', name='PR-AUC')
                    ],
    )

In [13]:
from keras.utils import to_categorical
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
y_val_cat = to_categorical(y_val)

Using TensorFlow backend.


## Model Training

In [14]:
model_path = '../classification_model_2_classes'
history = model.fit(X_train,y_train_cat, validation_data=(X_val, y_val_cat), epochs=100, callbacks=[reduce_lr])
tf.keras.models.save_model(model = model, filepath = model_path)

Train on 483 samples, validate on 69 samples
Epoch 1/100
483/483 [==============================] - 14s 29ms/sample - loss: 0.7112 - ROC-AUC: 0.5332 - PR-AUC: 0.5126 - val_loss: 0.7675 - val_ROC-AUC: 0.4033 - val_PR-AUC: 0.4225
Epoch 2/100
483/483 [==============================] - 6s 12ms/sample - loss: 0.6822 - ROC-AUC: 0.5925 - PR-AUC: 0.5768 - val_loss: 0.7336 - val_ROC-AUC: 0.4500 - val_PR-AUC: 0.4538
Epoch 3/100
483/483 [==============================] - 6s 12ms/sample - loss: 0.6647 - ROC-AUC: 0.6375 - PR-AUC: 0.6208 - val_loss: 0.7190 - val_ROC-AUC: 0.4789 - val_PR-AUC: 0.4806
Epoch 4/100
483/483 [==============================] - 6s 12ms/sample - loss: 0.6482 - ROC-AUC: 0.6818 - PR-AUC: 0.6656 - val_loss: 0.7075 - val_ROC-AUC: 0.5090 - val_PR-AUC: 0.5002
Epoch 5/100
483/483 [==============================] - 6s 12ms/sample - loss: 0.6409 - ROC-AUC: 0.7010 - PR-AUC: 0.6888 - val_loss: 0.7064 - val_ROC-AUC: 0.5177 - val_PR-AUC: 0.4990
Epoch 6/100
483/483 [=======================

In [15]:
X_test.shape

(139, 320, 320, 3)

In [16]:
from sklearn.metrics import classification_report,confusion_matrix, roc_auc_score

model = tf.keras.models.load_model(filepath = model_path)
predictions = model.predict(X_test)
predictions_rounded = np.argmax(predictions, axis=1)

In [17]:
list(set(predictions_rounded))

[0, 1]

In [18]:
from sklearn.metrics import roc_curve, auc

for p in list(set(predictions_rounded)):
    fpr, tpr, thresholds = roc_curve(y_test, model.predict(X_test)[:,p], pos_label = p)
    auroc = round(auc(fpr, tpr), 2)
    print ('{} - {}'.format(p, auroc))

0 - 0.86
1 - 0.86


In [19]:
confusion_matrix(y_test, predictions_rounded)

array([[74,  8],
       [19, 38]])

In [20]:
print (classification_report(y_test, predictions_rounded))

              precision    recall  f1-score   support

           0       0.80      0.90      0.85        82
           1       0.83      0.67      0.74        57

    accuracy                           0.81       139
   macro avg       0.81      0.78      0.79       139
weighted avg       0.81      0.81      0.80       139



## LIME Model Interpretation

In [ ]:
import lime
from lime import lime_range

In [ ]:
i = 0 # please change the value with the index of image in the dataframe
filepath = train_data_dir+metadata_df['filename'][i]
image = tf.keras.preprocessing.image.load_img(filepath, color_mode='rgb', target_size=(img_height, img_width))
image = np.array(image)

In [ ]:
explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(image, model.predict, top_labels=5)

In [ ]:
from skimage.segmentation import mark_boundaries

temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mark_boundaries(image, mask))

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[1], positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mark_boundaries(image, mask))

## Grad Cams

In [ ]:
# https://keras.io/examples/vision/grad_cam/
def get_img_array(image_path, image_size):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=image_size)
    array = tf.keras.preprocessing.image.img_to_array(image)
    array = np.expand_dims(array, axis=0)
    return (array)

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
        
    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return (heatmap.numpy())

def save_and_display_gradcam(img_path, heatmap, cam_path='./cam.jpg', alpha=0.4):
    img = tf.keras.preprocessing.image.load_img(img_path)
    img = tf.keras.preprocessing.image.img_to_array(img)
    
    heatmap = np.uint8(255*heatmap)
    jet = cm.get_cmap('jet')
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]
    
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)
    
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)
    superimposed_img.save(cam_path)
    
    display(Image(cam_path))

In [ ]:
i = 100 # please change the value with the index of image in the dataframe
image_path = train_data_dir+metadata_df[filename][i]
img_array = get_img_array(image_path=image_path, image_size=(img_height, img_width))
model = tf.keras.models.load_model(filepath = model_path)
model.layers[-1] = None
preds = model.predict(img_array)
heatmap = make_gradcam_heatmap(img_array, model, 
                               last_conv_layer_name='stage4_unit3_conv3')

save_and_display_gradcam(image_path, heatmap)